# 영상의 크기변환과 회전


## cv2.resize()


## cv2.rotate()

## cv2.getRotationMatrix2D함수와 cv2.warpAffine()함수

###

### 2D 변환 행렬 유도

#### 원점(0 , 0)을 기준으로 회전할 때 변환행렬 유도

  * 2D 평면에서 점 $(x, y)$를 원점 기준으로 $\theta$ 각도만큼 회전변환. (반시계)

  * 회전 변환 후의 좌표 $(x', y')$는 다음과 같이 주어집니다:
  $$
  x' = x \cos(\theta) - y \sin(\theta)
  $$
  $$
  y' = x \sin(\theta) + y \cos(\theta)
  $$

  > 극좌표계를 이용한 2D 좌표변환 행렬 설명
  * URL : https://www.youtube.com/watch?v=TEiwQ9v16UU


  * 이를 행렬 형태로 표현하면 다음과 같습니다:
  $$
  \begin{bmatrix}
  x' \\
  y'
  \end{bmatrix}
  =
  \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) \\
  \sin(\theta) & \cos(\theta)
  \end{bmatrix}
  \begin{bmatrix}
  x \\
  y
  \end{bmatrix}
  $$

---

  * 평행 이동은 각 좌표에 일정한 값을 더해주는 변환입니다.

  * 평행 이동 후의 좌표 $(x', y')$는 다음과 같이 주어집니다. (원래 좌표계에서 평행이동임):
  $$
  x' = x + t_x
  $$
  $$
  y' = y + t_y
  $$

  * 이를 행렬 형태로 표현하면 다음과 같습니다:
  $$
  \begin{bmatrix}
  x' \\
  y'
  \end{bmatrix}
  =
  \begin{bmatrix}
  1 & 0 \\
  0 & 1
  \end{bmatrix}
  \begin{bmatrix}
  x \\
  y
  \end{bmatrix}
  +
  \begin{bmatrix}
  t_x \\
  t_y
  \end{bmatrix}
  $$

---

  * 회전 변환과 평행 이동 변환을 결합하면 다음과 같은 변환을 얻게 됩니다:
  $$
  \begin{bmatrix}
  x' \\
  y'
  \end{bmatrix}
  =
  \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) \\
  \sin(\theta) & \cos(\theta)
  \end{bmatrix}
  \begin{bmatrix}
  x \\
  y
  \end{bmatrix}
  +
  \begin{bmatrix}
  t_x \\
  t_y
  \end{bmatrix}
  $$

  $$
  \downarrow
  $$

  $$
  \begin{bmatrix}
  x' \\
  y'
  \end{bmatrix}
  =
  \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) & t_x \\
  \sin(\theta) & \cos(\theta) & t_y
  \end{bmatrix}
  \begin{bmatrix}
  x \\
  y \\
  1
  \end{bmatrix}
  $$

  * 이 변환 행렬을 $M$이라고 하면, $M$은 다음과 같습니다:
  $$
  M = \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) & t_x \\
  \sin(\theta) & \cos(\theta) & t_y
  \end{bmatrix}
  $$

  $M$변환행렬을 이용하여 점 $(x, y)$를 변환하면 새로운 점 $(x', y')$를 얻을 수 있습니다.

---

  #### 2. 임의의 점$(c_x , c_y)$을 중심으로 회전할 때 변환행렬 유도

  * 임의의 점 $(c_x, c_y)$를 중심으로 회전하려면, 먼저 이 중심점으로 이동한 후 회전하고, 다시 원래 위치로 이동하는 과정이 필요합니다.

  1. 주어진 점 $(x, y)$에서 $(c_x, c_y)$를 원점으로 하는 새로운 좌표계로 변환.
    - $ (New_x + C_x = Old_x, Old_x - C_x = New_x) $

    - $(x', y'){new} = (x' + C_x, y' + C_y){old}$

  2. 회전
  3. 원래 좌표계로 변환

  * 이 변환 과정을 하나의 행렬로 결합하면 다음과 같습니다:
  $$
  \begin{aligned}
  M &= \text{Translate}(-c_x, -c_y) \cdot \text{Rotate}(\theta) \cdot \text{Translate}(c_x, c_y) \\
  &= \begin{bmatrix}
  1 & 0 & -c_x \\
  0 & 1 & -c_y \\
  0 & 0 & 1
  \end{bmatrix}
  \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) & 0 \\
  \sin(\theta) & \cos(\theta) & 0 \\
  0 & 0 & 1
  \end{bmatrix}
  \begin{bmatrix}
  1 & 0 & c_x \\
  0 & 1 & c_y \\
  0 & 0 & 1
  \end{bmatrix}
  \end{aligned}
  $$

  $$
  \downarrow
  $$

  $$
  M = \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) & c_x (1 - \cos(\theta)) + c_y \sin(\theta) \\
  \sin(\theta) & \cos(\theta) & c_y (1 - \cos(\theta)) - c_x \sin(\theta) \\
  0 & 0 & 1
  \end{bmatrix}
  $$

  * 여기서 마지막 행과 열은 동차 좌표계에서 사용되므로 OpenCV에서는 생략하여 2x3 형태의 행렬로 사용합니다:
  $$
  M = \begin{bmatrix}
  \cos(\theta) & -\sin(\theta) & c_x (1 - \cos(\theta)) + c_y \sin(\theta) \\
  \sin(\theta) & \cos(\theta) & c_y (1 - \cos(\theta)) - c_x \sin(\theta)
  \end{bmatrix}
  $$

  * 이렇게 하면 원점이 아닌 임의의 점을 중심으로 회전할 수 있으며, 추가적인 평행 이동 $(t_x, t_y)$를 고려할 경우:
  $$
  M[:, 2] += (t_x, t_y)
  $$

